# Figure 8: Glacier Signature

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr 
import geopandas as gpd
from shapely.geometry import Polygon

import os
from glob import glob
from oggm import utils

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia/MS2 Results/')

## Random forest results

In [ ]:
data_RF = pd.read_csv("RF_importance_basin.csv")
data_RF = data_RF.replace({"Glacier_Signature": {"Rate_change" : 'Rate of change'}})
data_RF = data_RF.drop("Unnamed: 0", axis=1) 

data_RF['best'] = data_RF[["Outline","Climate", "Volume", "GGM", "SSP", "BCM"]].idxmax(axis=1)
data_RF = data_RF.rename(columns = {"Zone": "ID"})

data_RF.ID = data_RF.ID.str[1:] # trick ID code
data_RF["ID"] = data_RF.ID.astype("int64")

In [ ]:
# new df with the most important variable for each basin and metric (column)
rf_timing      = data_RF[data_RF["Glacier_Signature"] == "Timing"][[     "ID", "best"]].set_index("ID")
rf_frequency   = data_RF[data_RF["Glacier_Signature"] == "Frequency"][[  "ID", "best"]].set_index("ID")
rf_duration    = data_RF[data_RF["Glacier_Signature"] == "Duration"][[   "ID", "best"]].set_index("ID")
rf_magnitude   = data_RF[data_RF["Glacier_Signature"] == "Magnitude"][[  "ID", "best"]].set_index("ID")
rf_rate_change = data_RF[data_RF["Glacier_Signature"] == "Rate of change"][["ID", "best"]].set_index("ID")

data_RF_new = pd.concat([rf_timing, rf_frequency, rf_duration, rf_magnitude, rf_rate_change], axis=1)
data_RF_new.columns = ['Timing', 'Frequency', 'Duration', 'Magnitude', 'Rate of change']

##  Basin polygons

In [ ]:
# merge gpd and RF dataset
basins = gpd.read_file("/home/rooda/Dropbox/Patagonia/GIS South/Basins_Patagonia_ice.shp")
basins["ID"] = basins.ID.astype("int64")
basins = basins.set_index("ID")

names = ["Yelcho", "Baker", "Santa Cruz                           ", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"]
basins.loc[basins.basin_area > 5000, "Name"] = names

basins = pd.merge(basins, data_RF_new, left_index=True, right_index=True)
basins # only 73 because we need to update the runs 

In [ ]:
# more date for the base map
geo_map = gpd.read_file("/home/rooda/Dropbox/ArcGIS/Chile/south_america.shp")
geo_map = geo_map[(geo_map.CC == "CI") | (geo_map.CC == "AR")]
geo_map = geo_map.dissolve(by='REGION')
geo_map["geometry"] = geo_map.simplify(0.01)
poly_gdf = Polygon([(-74.98, -55.7), (-74.98, -40.52), (-68.05, -40.52), (-68.05, -55.7), (-74.98, -55.8)])
poly_gdf = gpd.GeoDataFrame([1], geometry=[poly_gdf], crs=geo_map.crs)

geo_map = geo_map.clip(poly_gdf)

In [ ]:
fig = make_subplots(rows=2, cols=5, horizontal_spacing = 0.01, vertical_spacing = 0.01, row_heights = [0.42,0.58], 
                    specs=[[{"type": "box", "colspan": 5}, None,                   None,                    None,                    None], 
                           [{"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}, {"type": "scattergeo"}]])

# a) Boxplot -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Box(x = data_RF.Glacier_Signature, y = data_RF.Outline/100, name = "Geometry",  boxmean=True, marker_color=cl[3], showlegend = False), row=1, col=1)
fig.add_trace(go.Box(x = data_RF.Glacier_Signature, y = data_RF.Volume/100,  name = "Volume",    boxmean=True, marker_color=cl[1], showlegend = False), row=1, col=1)
fig.add_trace(go.Box(x = data_RF.Glacier_Signature, y = data_RF.Climate/100, name = "Climate",   boxmean=True, marker_color=cl[2], showlegend = False), row=1, col=1)
fig.add_trace(go.Box(x = data_RF.Glacier_Signature, y = data_RF.GGM/100,     name = "GGM",       boxmean=True, marker_color=cl[0], showlegend = False), row=1, col=1)
fig.add_trace(go.Box(x = data_RF.Glacier_Signature, y = data_RF.SSP/100,     name = "SSP",       boxmean=True, marker_color=cl[4], showlegend = False), row=1, col=1)
fig.add_trace(go.Box(x = data_RF.Glacier_Signature, y = data_RF.BCM/100,     name = "BCM",       boxmean=True, marker_color=cl[5], showlegend = False), row=1, col=1)

fig.update_yaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", mirror=True,  griddash = "dot", zeroline = False, title='Importance (% IncMSE)', tickformat = ',.0%',row=1, col=1)
fig.update_xaxes(showline = True, linecolor = 'rgba(0,0,0,0.5)', linewidth = 1, ticks="outside", mirror=True,  griddash = "dot", tickfont = dict(size=16), row=1, col=1)
fig.add_annotation(text="a)", font = dict(size = 18), xref="paper", yref="paper", x = 0.01, y = 0.99, showarrow=False)
fig.update_layout(boxgroupgap=0.15, boxgap=0.2, boxmode='group', font=dict(size=14))

# maps -----------------------------------------------------------------------------------------------------------------
basins = basins.replace({"Outline":0, "Volume":1, "Climate":2, "GGM":3, "SSP":4, "BCM":5}) # trick for plotting using go.Choropleth (discrete data)

for x in range(1,6): 
    fig.add_trace(go.Choropleth(geojson = eval(geo_map['geometry'].to_json()),  locations = geo_map.index, z = geo_map['iso_num'],
                            colorscale = ["#d5d5d5", "#d5d5d5"], showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=x)

    fig.add_scattergeo(geojson = eval(basins['geometry'].to_json()), locations = basins.index, text = basins['Name'], mode = 'text', showlegend = False, 
                   textfont=dict(size=8, color = "rgba(255,255,255,0.7)"),row=2, col=x)
    
# b) Timing -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()),  locations = basins.index, z = basins['Timing'],
                            coloraxis="coloraxis", showscale= False, marker_line_color ='white', marker_line_width=0.1), row=2, col=1)
fig.add_annotation(text="b)", font = dict(size = 18), xref="paper", yref="paper", x = 0.01, y = 0.52, showarrow=False)

# c) Frequency -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['Frequency'], 
                            coloraxis="coloraxis", showscale= False, marker_line_color='white', marker_line_width=0.1), row=2, col=2)
fig.add_annotation(text="c)", font = dict(size = 18), xref="paper", yref="paper", x = 0.205, y = 0.52, showarrow=False)

# d) Duration -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['Duration'],
                            coloraxis="coloraxis", marker_line_color='white', marker_line_width=0.1, showscale= False), row=2, col=3)
fig.add_annotation(text="d)", font = dict(size = 18), xref="paper", yref="paper", x = 0.42, y = 0.52, showarrow=False)

# e) Magnitude -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['Magnitude'], 
                            coloraxis="coloraxis", marker_line_color='white', marker_line_width=0.1, showscale= False), row=2, col=4)
fig.add_annotation(text="e)", font = dict(size = 18), xref="paper", yref="paper", x = 0.62, y = 0.52, showarrow=False)

# f) Rate of change -----------------------------------------------------------------------------------------------------------
fig.add_trace(go.Choropleth(geojson = eval(basins['geometry'].to_json()), locations = basins.index, z = basins['Rate of change'], 
                            coloraxis="coloraxis",  marker_line_color='white', marker_line_width=0.1, showscale= False), row=2, col=5)
fig.add_annotation(text="f)", font = dict(size = 18), xref="paper", yref="paper", x = 0.83, y = 0.52, showarrow=False)

cl = [px.colors.qualitative.D3[3], px.colors.qualitative.D3[1], px.colors.qualitative.D3[2], px.colors.qualitative.D3[0], px.colors.qualitative.D3[4], px.colors.qualitative.D3[5]]
cc_scale = ([(0, cl[0])] + [(e, cl[(i + 1) // 2]) for i, e in enumerate(np.repeat(np.linspace(0, 1, 7)[1:6], 2))] + [(1, cl[5])])
colorbar  =  {"tickvals": [0,1,2,3,4,5], "ticktext": ["Outline", "Volume", "Climate", "GGM", "SSP", "BCM"], "ticklabelposition" : "outside", "len" : 0.9}
coloraxis =  {"colorscale": cc_scale, "colorbar": colorbar, "cmin" : -0.5, "cmax" : 5.5}

## Layout and geos
fig.update_geos(showframe = True, framewidth = 1, framecolor = "rgba(0,0,0,0.5)", lonaxis_range=[-75, -68], lataxis_range=[-55.8, -40.5], 
                bgcolor = "#f9f9f9", showland = False, showcoastlines = False, showlakes = False)
fig.update_layout(coloraxis = coloraxis, plot_bgcolor="rgba(213,213,213,0.7)")
fig.update_layout(autosize = False, width = 1250, height = 900, margin = dict(l=5, r=5, b=5, t=5))
fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_8_signature.png", scale=4)
fig.show()

## Figures for slides

In [ ]:
# data
data_timing    = pd.read_csv("data_peak_water_year.csv")
data_rate      = pd.read_csv("data_rate_change.csv")
data_duration  = pd.read_csv("data_duration.csv")
data_magnitude = pd.read_csv("data_peak_water_value.csv")
data_frequency = pd.read_csv("data_freq.csv")

In [ ]:
data = data_timing
fig  = make_subplots(rows=3, cols=3, horizontal_spacing = 0.04, vertical_spacing = 0.04, 
                       subplot_titles = zones.reshape(1,9).tolist()[0], shared_xaxes= True, shared_yaxes= True, 
                       x_title = "Peak water (year)", y_title = "Percent (%)")

for x in range(0,3):
    for y in range(0,3):
        fig.add_trace(go.Histogram(x = data[zones[x,y]].to_numpy(), histnorm='percent', 
                                   marker_color= cl[0], xbins=dict(start=1980, size=5, end=2100), 
                                   showlegend=False), row=x+1, col=y+1)
        
fig.update_layout(bargap=0.1, font=dict(size=14), barmode='overlay', template='seaborn', autosize=False, width=1100, height=700)

fig.update_xaxes(range= [1980,2100])
fig.update_yaxes(range= [0,50])
fig.update_traces(opacity=0.7)
fig.show()

In [ ]:
data = data_timing
fig  = make_subplots(rows=3, cols=3, horizontal_spacing = 0.04, vertical_spacing = 0.04, 
                       subplot_titles = zones.reshape(1,9).tolist()[0], shared_xaxes= True, shared_yaxes= True, 
                       x_title = "Peak water (year)", y_title = "Percent (%)")

for x in range(0,3):
    for y in range(0,3):
        fig.add_trace(go.Histogram(x = data[(data.SSP == "ssp126")][zones[x,y]], name = "SSP126", histnorm='percent', 
                                   marker_color= cl[0], xbins=dict(start=1980, size=5, end=2100), 
                                   showlegend=True), row=x+1, col=y+1)

        fig.add_trace(go.Histogram(x = data[(data.SSP == "ssp585")][zones[x,y]], name = "SSP585", histnorm='percent', 
                                   marker_color= cl[3], xbins=dict(start=1980, size=5, end=2100), 
                                   showlegend=True), row=x+1, col=y+1)
        
fig.update_layout(bargap = 0.1, font=dict(size=14), barmode='overlay', template='seaborn', autosize=False, width=1100, height=700)

fig.update_xaxes(range= [1980,2100])
fig.update_yaxes(range= [0,50])
fig.update_traces(opacity=0.7)
fig.show()